In [ ]:
!pip install langchain_community neo4j langchain langchain_groq sentence-transformers langchain_huggingface langchain_nvidia_ai_endpoints


In [5]:
import pandas as pd
df = pd.read_csv("filtered_results.csv")


In [3]:
!pip install langchain_community neo4j langchain langchain_groq sentence-transformers langchain_huggingface langchain_nvidia_ai_endpoints

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 825.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 21.2 MB/s eta 0:00:00


In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # to restricts its context window, most of model support larger, but make sure to use it based on your need since it will consumes more compute and VRAM
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [2]:
import pandas as pd
import numpy as np
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.chains import GraphCypherQAChain
from google.colab import userdata

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested 8, 16, 32, 64, 128, higher ranks can store more information but increase the computational and memory cost of LoRA.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # rule of thumb, double the r or equal. alpha is a scaling factor for updates
    lora_dropout = 0, # Supports any, but = 0 is optimized. Probability of zeroing out elements in low-rank matrices for regularization.
    random_state = 3407,
    use_rslora = True,  # has been proven to work better ( https://arxiv.org/pdf/2312.03732 )
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
df

,original_index,original_index.1,original_index.2,question,type,cypher,syntax_error,timeout,returns_results
0,1.0,1.0,0.0,What ResourceSource node has an Item named NuC...,Simple Retrieval Queries,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...,False,False,True
1,2.0,2.0,1.0,Which Item node has identifier SCR_004561 ?,Simple Retrieval Queries,MATCH (item:Item {identifier: 'SCR_004561'}) R...,False,False,True
2,3.0,3.0,2.0,What is the proper citation of RRID with the i...,Simple Retrieval Queries,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...,False,False,True
3,4.0,4.0,3.0,Return the first 3 mentions with 'timestamp' g...,Simple Retrieval Queries,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...,False,False,True
4,8.0,8.0,7.0,Which organization in 'HIERARCHY' is 'Parent' ...,Simple Retrieval Queries,MATCH (child:Organization {name: 'Brighter Ide...,False,False,True
5,10.0,10.0,9.0,What are the issues associated with ResourceSo...,Simple Retrieval Queries,MATCH (n:ResourceSource {name: 'TETRA'})-[]-(m...,False,False,True
6,12.0,12.0,11.0,Return all provenance which has primary source...,Simple Retrieval Queries,MATCH (a:ResourceSource)-[:HAS_PROVENANCE]->(p...,False,False,True
7,17.0,17.0,16.0,What are the SCORE of RRID with identifier SCR...,Simple Retrieval Queries,MATCH (item:RRID {curie: 'SCR_004689'}) RETURN...,False,False,True
8,18.0,18.0,17.0,Return top 3 Item node with identifier SCR_004...,Simple Retrieval Queries,MATCH (n:Item {identifier: 'SCR_004561'}) RETU...,False,False,True
9,24.0,24.0,23.0,Can you list the keywords of all Item nodes in...,Complex Retrieval Queries,MATCH (o:Organism)<-[:IS_RELATED_TO]-(r:Resour...,False,False,True


In [19]:
df = df[['question','cypher']]
df.rename(columns={'question': 'input','cypher':'output'}, inplace=True)
df.reset_index(drop=True, inplace=True)
df

<ipython-input-19-ed1885fb552d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'question': 'input','cypher':'output'}, inplace=True)


,input,output
0,What ResourceSource node has an Item named NuC...,MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item)...
1,Which Item node has identifier SCR_004561 ?,MATCH (item:Item {identifier: 'SCR_004561'}) R...
2,What is the proper citation of RRID with the i...,MATCH (rrid:RRID {curie: 'RRID:SCR_004561'}) R...
3,Return the first 3 mentions with 'timestamp' g...,MATCH (n:Mentions) WHERE n.timestamp > '2025-0...
4,Which organization in 'HIERARCHY' is 'Parent' ...,MATCH (child:Organization {name: 'Brighter Ide...
5,What are the issues associated with ResourceSo...,MATCH (n:ResourceSource {name: 'TETRA'})-[]-(m...
6,Return all provenance which has primary source...,MATCH (a:ResourceSource)-[:HAS_PROVENANCE]->(p...
7,What are the SCORE of RRID with identifier SCR...,MATCH (item:RRID {curie: 'SCR_004689'}) RETURN...
8,Return top 3 Item node with identifier SCR_004...,MATCH (n:Item {identifier: 'SCR_004561'}) RETU...
9,Can you list the keywords of all Item nodes in...,MATCH (o:Organism)<-[:IS_RELATED_TO]-(r:Resour...


In [7]:
neo4j_url = "neo4j+s://a38c58ba.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "TnAlTgGxuAsAV7UsD0FbRgoKKSmBuBSekNyGmvzJG5Y"

graph = Neo4jGraph(
    url=neo4j_url,
    username=neo4j_user,
    password=neo4j_password,
    sanitize=True,
    enhanced_schema=True
)

<ipython-input-7-620215b8bbe4>:5: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [11]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **ResourceSource**
  - `uid`: STRING Available options: ['ResourceSource_ee3bcd00-1c94-471d-9e9e-6034315274d', 'ResourceSource_a90bf7bb-a924-4c8d-9429-c4a7dd5f33e', 'ResourceSource_c880a4ed-3b3a-4d50-ab37-c8a6388ed9b', 'ResourceSource_a131b410-737e-4a43-ae63-a48abd8c453', 'ResourceSource_a6bdf60f-12ea-4925-9955-196089fdef0', 'ResourceSource_285842d0-173c-4633-adea-f4915f1fa8a', 'ResourceSource_8551020e-aa37-460c-845a-3182b166dc4', 'ResourceSource_6b7ce365-752c-420a-9fc5-5fe71cff110', 'ResourceSource_9f62b9ae-b15a-41bc-a6ac-bb08194d87c', 'ResourceSource_f48ac10a-514f-4a09-aadf-95a61e14f6d']
  - `label`: STRING Available options: ['ResourceSource']
  - `recordValid`: BOOLEAN 
- **Item**
  - `uid`: STRING Available options: ['Item_3be8ecbc-9b0e-4b5a-b187-42d30120eea1', 'Item_dc253cee-a54f-4b53-a79b-77e601b40efb', 'Item_b6b7e72a-e557-4230-8e9c-ed162a4381d9', 'Item_dd492de4-ea25-4e5b-a655-9ac75f07b962', 'Item_a5f9ec66-b512-4a2a-9f15-b7a1ea8dd235', 'Item_87975754-75f5-4f52

In [29]:
from neo4j import GraphDatabase

uri = "neo4j+s://a38c58ba.databases.neo4j.io"
user = "neo4j"
password = "TnAlTgGxuAsAV7UsD0FbRgoKKSmBuBSekNyGmvzJG5Y"

driver = GraphDatabase.driver(uri, auth=(user, password))

def get_schema_summary(tx):
    schema = []

    # Node labels and properties
    node_query = """
    MATCH (n)
    WITH labels(n) AS labels, keys(n) AS props
    UNWIND labels AS label
    UNWIND props AS prop
    RETURN DISTINCT label, prop
    ORDER BY label, prop
    """

    # Full relationship info: source → rel → target + props
    rel_query = """
    MATCH (a)-[r]->(b)
    UNWIND labels(a) AS fromLabel
    UNWIND labels(b) AS toLabel
    RETURN DISTINCT fromLabel, type(r) AS relType, toLabel, keys(r) AS relProps
    ORDER BY fromLabel, relType, toLabel
    """

    nodes = tx.run(node_query).data()
    rels = tx.run(rel_query).data()

    from collections import defaultdict

    node_map = defaultdict(set)
    for row in nodes:
        node_map[row['label']].add(row['prop'])
    for label, props in node_map.items():
        schema.append(f"Node: {label}, Properties: {', '.join(sorted(props))}")

    for row in rels:
        from_label = row['fromLabel']
        rel_type = row['relType']
        to_label = row['toLabel']
        props = row['relProps']
        prop_str = ", Properties: " + ", ".join(sorted(props)) if props else ""
        schema.append(f"Relationship: ({from_label})-[:{rel_type}]->({to_label}){prop_str}")

    return "\n".join(schema)

with driver.session() as session:
    schema_summary = session.execute_read(get_schema_summary)

print("📘 Extracted Schema from Neo4j:\n")
print(schema_summary)


📘 Extracted Schema from Neo4j:

Node: Abbreviation, Properties: label, name, uid
Node: Agency, Properties: identifier, label, name, uid
Node: AlternateIdentifier, Properties: identifier, label, uid
Node: Authority, Properties: label, name, uid
Node: Availability, Properties: description, keyword, label, uid
Node: Availability2, Properties: keyword, label, uid
Node: ChildRelationship, Properties: label, uid
Node: Chunk, Properties: embedding, id, query, question, text
Node: ContentType, Properties: curie, label, name, uid
Node: CountValue, Properties: count, label, uid
Node: DataItem, Properties: label, uid
Node: Disco, Properties: label, uid, v_uuid
Node: Disease, Properties: label, name, role, uid
Node: Diseases, Properties: label, uid
Node: Distributions, Properties: label, uid
Node: Graph, Properties: label, uid
Node: Hierarchy, Properties: label, uid
Node: Issues, Properties: label, uid
Node: Item, Properties: curie, description, docid, identifier, label, language, name, uid, uuid


In [30]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = f"Convert text to cypher query based on this schema: {schema_summary}" # examples["instructions"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [31]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 44
})

In [32]:
dataset[0]

{'input': 'What ResourceSource node has an Item named NuChart ?',
 'output': "MATCH (r:ResourceSource)-[:HAS_ITEM]->(i:Item) WHERE i.name='NuChart' RETURN r.uid",
 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert text to cypher query based on this schema: Node: Abbreviation, Properties: label, name, uid\nNode: Agency, Properties: identifier, label, name, uid\nNode: AlternateIdentifier, Properties: identifier, label, uid\nNode: Authority, Properties: label, name, uid\nNode: Availability, Properties: description, keyword, label, uid\nNode: Availability2, Properties: keyword, label, uid\nNode: ChildRelationship, Properties: label, uid\nNode: Chunk, Properties: embedding, id, query, question, text\nNode: ContentType, Properties: curie, label, name, uid\nNode: CountValue, Properties: count, label, uid\nNode: DataItem, Properties: label, uid\nNode

In [41]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs=3,
        learning_rate = 2e-4,  # the rate at which the model updates its parameters during training.
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/44 [00:00<?, ? examples/s]

In [42]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 44 | Num Epochs = 3 | Total steps = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,0.383500
2,0.354400
3,0.328200
4,0.278600
5,0.190200
6,0.117600
7,0.125800
8,0.096800
9,0.067700
10,0.071700


In [44]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema, just give the cypher query: {schema_summary}", # instruction
        "give item.name whose rrid = scr_004689", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert text to cypher query based on this schema, just give the cypher query: Node: Abbreviation, Properties: label, name, uid
Node: Agency, Properties: identifier, label, name, uid
Node: AlternateIdentifier, Properties: identifier, label, uid
Node: Authority, Properties: label, name, uid
Node: Availability, Properties: description, keyword, label, uid
Node: Availability2, Properties: keyword, label, uid
Node: ChildRelationship, Properties: label, uid
Node: Chunk, Properties: embedding, id, query, question, text
Node: ContentType, Properties: curie, label, name, uid
Node: CountValue, Properties: count, label, uid
Node: DataItem, Properties: label, uid
Node: Disco, Properties: label, uid, v_uuid
Node: Disease, Properties: label, name, role, uid
Node: Diseases, Properties: label, uid
Node: Distrib

In [45]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema, just give the cypher query: {schema_summary}", # instruction
        "give rrid of item nuchart", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert text to cypher query based on this schema, just give the cypher query: Node: Abbreviation, Properties: label, name, uid
Node: Agency, Properties: identifier, label, name, uid
Node: AlternateIdentifier, Properties: identifier, label, uid
Node: Authority, Properties: label, name, uid
Node: Availability, Properties: description, keyword, label, uid
Node: Availability2, Properties: keyword, label, uid
Node: ChildRelationship, Properties: label, uid
Node: Chunk, Properties: embedding, id, query, question, text
Node: ContentType, Properties: curie, label, name, uid
Node: CountValue, Properties: count, label, uid
Node: DataItem, Properties: label, uid
Node: Disco, Properties: label, uid, v_uuid
Node: Disease, Properties: label, name, role, uid
Node: Diseases, Properties: label, uid
Node: Distrib